In [9]:
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Sense, Options
import sys

Tenemos:

- 3 fábricas con capacidades de producción: F1 (100 unidades), F2 (150 unidades), F3 (200 unidades)
- 4 clientes con demandas: C1 (120 unidades), C2 (80 unidades), C3 (100 unidades), C4 (150 unidades)
- Costos de transporte (por unidad) desde cada fábrica a cada cliente:

|    | C1      | C2      | C3      | C4      |
| -- | ------- | ------- | ------- | ------- |
| F1 | 5       | 3       | 6       | 2       |
| F2 | 4       | 7       | 5       | 8       |
| F3 | 6       | 4       | 3       | 5       |

In [2]:
# 1. Crear Contenedor
container = Container()

In [3]:
# 2. Definir Conjuntos
factories = Set(container=container, name="factories", records=["F1", "F2", "F3"])
clients = Set(container=container, name="clients", records=["C1", "C2", "C3", "C4"])

factories, clients

(Set(name='factories', domain=['*']), Set(name='clients', domain=['*']))

In [4]:
# 3. Definir Parametros
offer = Parameter(container=container, name='offer', domain=[
                  factories], records=[['F1', 100], ['F2', 150], ['F3', 200]])

demand = Parameter(container=container, name='demand', domain=[clients], records=[
                   ['C1', 120], ['C2', 80], ['C3', 100], ['C4', 150]])

cost = Parameter(container=container, name='cost', domain=[factories, clients], records=[
    ['F1', 'C1', 5], ['F1', 'C2', 3], ['F1', 'C3', 6], ['F1', 'C4', 2],
    ['F2', 'C1', 4], ['F2', 'C2', 7], ['F2', 'C3', 5], ['F2', 'C4', 8],
    ['F3', 'C1', 6], ['F3', 'C2', 4], ['F3', 'C3', 3], ['F3', 'C4', 5],
])

offer, demand, cost

(Parameter(name='offer', domain=[Set(name='factories', domain=['*'])]),
 Parameter(name='demand', domain=[Set(name='clients', domain=['*'])]),
 Parameter(name='cost', domain=[Set(name='factories', domain=['*']), Set(name='clients', domain=['*'])]))

In [5]:
# 4. Definir Variables
x = Variable(container=container, name='x', domain=[
             factories, clients], type='Positive')

x

Variable(name='x', domain=[Set(name='factories', domain=['*']), Set(name='clients', domain=['*'])], type='positive')

In [6]:
# 5. Definir Equaciones
supply_equation = Equation(container=container, name='supply_equation', domain=factories)
demand_equation = Equation(container=container, name='demand_equation', domain=clients)

supply_equation[factories] = Sum(clients, x[factories, clients]) <= offer[factories]
demand_equation[clients] = Sum(factories, x[factories, clients]) >= demand[clients]
objective = Sum((factories, clients), cost[factories, clients] * x[factories, clients])

supply_equation, demand_equation, objective

(Equation(name='supply_equation', type='eq', domain=[Set(name='factories', domain=['*'])]),
 Equation(name='demand_equation', type='eq', domain=[Set(name='clients', domain=['*'])]),
 Sum(domain=[], expression=Expression(left=ImplicitParameter(parent=Parameter(name='cost', domain=[Set(name='factories', domain=['*']), Set(name='clients', domain=['*'])]), name='cost', domain=[Set(name='factories', domain=['*']), Set(name='clients', domain=['*'])], permutation=None), parent_scalar_domains=[]), data=*, right=ImplicitVariable(parent=Variable(name='x', domain=[Set(name='factories', domain=['*']), Set(name='clients', domain=['*'])], type='positive'), name='x', domain=[Set(name='factories', domain=['*']), Set(name='clients', domain=['*'])], permutation=None, parent_scalar_domains=[]))))

In [7]:
# 6. Crear el Modelo
transport = Model(container=container, name='transport', equations=container.getEquations(
), problem='LP', sense=Sense.MIN, objective=objective)

In [10]:
# 7. Solucionar
transport.solve(output=sys.stdout, options=Options(equation_listing_limit=10, variable_listing_limit=10))

--- _868f2047-a5da-41ec-8add-2919448bf915.gms(137) 4 Mb
--- Job _868f2047-a5da-41ec-8add-2919448bf915.gms Start 03/31/25 17:57:23 49.3.0 7de46a92 LEX-LEG x86 64bit/Linux
--- Applying:
    /home/arch/dev/python/gamspy/.venv/lib/python3.13/site-packages/gamspy_base/gmsprmun.txt
--- GAMS Parameters defined
    LP CPLEX
    Input /tmp/tmppsm_e54i/_868f2047-a5da-41ec-8add-2919448bf915.gms
    Output /tmp/tmppsm_e54i/_868f2047-a5da-41ec-8add-2919448bf915.lst
    ScrDir /tmp/tmppsm_e54i/tmp00a4y3w0/
    SysDir /home/arch/dev/python/gamspy/.venv/lib/python3.13/site-packages/gamspy_base/
    LogOption 3
    Trace /tmp/tmppsm_e54i/_868f2047-a5da-41ec-8add-2919448bf915.txt
    License /home/arch/dev/python/gamspy/.venv/lib/python3.13/site-packages/gamspy_base/gamslice.txt
    OptDir /tmp/tmppsm_e54i/
    LimRow 10
    LimCol 10
    TraceOpt 3
    GDX /tmp/tmppsm_e54i/_868f2047-a5da-41ec-8add-2919448bf915out.gdx
    SolPrint 0
    PreviousWork 1
    gdxSymbols newOrChanged
Licensee: GAMS Demo, for

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,1610,8,13,LP,CPLEX,0.001


In [11]:
transport.getEquationListing()

'supply_equation(F1)..  x(F1,C1) + x(F1,C2) + x(F1,C3) + x(F1,C4) =L= 100 ; (LHS = 100)\nsupply_equation(F2)..  x(F2,C1) + x(F2,C2) + x(F2,C3) + x(F2,C4) =L= 150 ; (LHS = 150)\nsupply_equation(F3)..  x(F3,C1) + x(F3,C2) + x(F3,C3) + x(F3,C4) =L= 200 ; (LHS = 200)\ndemand_equation(C1)..  x(F1,C1) + x(F2,C1) + x(F3,C1) =G= 120 ; (LHS = 120)\ndemand_equation(C2)..  x(F1,C2) + x(F2,C2) + x(F3,C2) =G= 80 ; (LHS = 80)\ndemand_equation(C3)..  x(F1,C3) + x(F2,C3) + x(F3,C3) =G= 100 ; (LHS = 100)\ndemand_equation(C4)..  x(F1,C4) + x(F2,C4) + x(F3,C4) =G= 150 ; (LHS = 150)\ntransport_objective..  5*x(F1,C1) + 3*x(F1,C2) + 6*x(F1,C3) + 2*x(F1,C4) + 4*x(F2,C1) + 7*x(F2,C2) + 5*x(F2,C3) + 8*x(F2,C4) + 6*x(F3,C1) + 4*x(F3,C2) + 3*x(F3,C3) + 5*x(F3,C4) - transport_objective_variable =E= 0 ; (LHS = 0)'

In [12]:
transport.getVariableListing()

'x(F1,C1)\n                (.LO, .L, .UP, .M = 0, 0, +INF, 6)\n        1       supply_equation(F1)\n        1       demand_equation(C1)\n        5       transport_objective\n\nx(F1,C2)\n                (.LO, .L, .UP, .M = 0, 0, +INF, 2)\n        1       supply_equation(F1)\n        1       demand_equation(C2)\n        3       transport_objective\n\nx(F1,C3)\n                (.LO, .L, .UP, .M = 0, 0, +INF, 6)\n        1       supply_equation(F1)\n        1       demand_equation(C3)\n        6       transport_objective\n\nx(F1,C4)\n                (.LO, .L, .UP, .M = 0, 100, +INF, 0)\n        1       supply_equation(F1)\n        1       demand_equation(C4)\n        2       transport_objective\n\nx(F2,C1)\n                (.LO, .L, .UP, .M = 0, 120, +INF, 0)\n        1       supply_equation(F2)\n        1       demand_equation(C1)\n        4       transport_objective\n\nx(F2,C2)\n                (.LO, .L, .UP, .M = 0, 0, +INF, 1)\n        1       supply_equation(F2)\n        1       deman

In [13]:
transport.objective_value

1610.0

In [14]:
transport.toLatex(path='latex', generate_pdf=True)

[CONVERTER - INFO] LaTeX (.tex) file has been generated under latex/transport.tex


pdfTeX 3.141592653-2.6-1.40.27 (TeX Live 2025/Arch Linux)
kpathsea version 6.4.1
Copyright 2025 Han The Thanh (pdfTeX) et al.
There is NO warranty.  Redistribution of this software is
covered by the terms of both the pdfTeX copyright and
the Lesser GNU General Public License.
For more information about these matters, see the file
named COPYING and the pdfTeX source.
Primary author of pdfTeX: Han The Thanh (pdfTeX) et al.
Compiled with libpng 1.6.47; using libpng 1.6.47
Compiled with zlib 1.3.1; using zlib 1.3.1
Compiled with xpdf version 4.04


In [15]:
transport.toGams(path='gams')

[CONVERTER - INFO] GAMS (.gms) file has been generated under gams/transport.gms


In [22]:
x.records.pivot(index='factories', columns='clients', values='level')

clients,C1,C2,C3,C4
factories,,,,
F1,0.0,0.0,0.0,100.0
F2,120.0,0.0,30.0,0.0
F3,0.0,80.0,70.0,50.0
